In this notebook we will generate C code and use CVode from the sundials suite to integrate the system of differential equations. Sundials is a well established and well tested open-source BSD-licensed library with excellent documentation. [Here is the documentation for CVode](https://computation.llnl.gov/sites/default/files/public/cv_guide.pdf).

We will use a thin Cython wrapper which calls the integration routine and returns the solution as a numpy vector and a dictionary, along with information about the integration.

In [ ]:
import json
import time
import numpy as np
from odesys import ODEsys
from chem import odesys_from_reactions_names_and_params

Again, using the `ODEsys` convenience class from notebook "35":

In [ ]:
watrad_data = json.load(open('radiolysis_300_Gy_s.json'))
watrad = odesys_from_reactions_names_and_params(**watrad_data)
tout = np.logspace(-6, 3, 200)  # close to one hour of operation
c0 = {'H2O': 55.4e3, 'H+': 1e-4, 'OH-': 1e-4}
y0 = [c0.get(symb.name, 0) for symb in watrad.y]

In [ ]:
tic = time.time()
yout, info = watrad.integrate_odeint(tout, y0)
toc = time.time()
print("Time of integration: {0:.1f} ms".format(1e3*(toc - tic)))

that is still the benchmark to beat. Subclassing `ODEsys` to have it render, compile and import the code:

In [ ]:
# %load cvode.py
import os
import uuid
import sympy as sp
import setuptools
import numpy as np
import pyximport
from odesys import ODEsys

lapack_libs = ['openblas']
pyximport.install()
setup_args={
    'include_dirs': [os.getcwd()],
    'libraries': lapack_libs + ['sundials_cvode', 'sundials_nvecserial', 'm']
}

pyxbld_template = """
def make_ext(modname, pyxfilename):
    from setuptools import Extension
    return Extension(
        name=modname,
        sources=[pyxfilename],
        include_dirs=%(include_dirs)s,
        libraries=%(libraries)s
    )
"""

class ODEcvode(ODEsys):

    def setup(self):
        self.uid = uuid.uuid4().hex[:10]
        self.mod_name = 'ode_c_%s' % self.uid
        idxs = list(range(len(self.f)))
        subs = {s: sp.Symbol('y[%d]' % i) for i, s in enumerate(self.y)}
        f_exprs = ['out[%d] = %s;' % (i, sp.ccode(self.f[i].xreplace(subs)))
                   for i in idxs]
        j_col_defs = ['realtype * const col_%d = DENSE_COL(J, %d);' % (ci, ci)
                      for ci in idxs]
        j_exprs = ['col_%d[%d] = %s;' % (ci, ri, self.j[ri, ci].xreplace(subs))
                   for ci in idxs for ri in idxs if self.j[ri, ci] != 0]
        ctx = dict(
            func = '\n    '.join(f_exprs + ['return 0;']),
            dense_jac = '\n    '.join(j_col_defs + j_exprs + ['return 0;']),
            band_jac = 'return -1;'
        )
        cvode_template = open(os.path.join('sundials_templates', 'integrate_serial.c')).read()
        cython_template = open(os.path.join('sundials_templates', '_integrate_serial.pyx')).read()
        open('integrate_serial_%s.c' % self.uid, 'wt').write(cvode_template % ctx)
        open('%s.pyx' % self.mod_name, 'wt').write(cython_template % {'uid': self.uid})
        open('%s.pyxbld' % self.mod_name, 'wt').write(pyxbld_template % {k: str(v) for k, v in setup_args.items()})
        self.mod = __import__(self.mod_name)
        self.integrate_odeint = None

    def integrate(self, tout, y0, params=(), rtol=1e-8, atol=1e-8, **kwargs):
        return self.mod._integrate(np.asarray(tout, dtype=np.float64),
                                   np.asarray(y0, dtype=np.float64),
                                   np.atleast_1d(np.asarray(params, dtype=np.float64)),
                                   abstol=np.atleast_1d(np.asarray(atol, dtype=np.float64)),
                                   reltol=rtol,
                                   **kwargs)


In [ ]:
cvode_sys = odesys_from_reactions_names_and_params(ODEcls=ODEcvode, **watrad_data)

In [ ]:
tic = time.time()
yout, info = cvode_sys.integrate(tout, y0)
toc = time.time()
print("Time of integration: {0:.1f} ms".format(1e3*(toc - tic)))

That is a considerable speed up from before. But the solver still has to
allocate memory for creating new arrays at each call, and each evaluation
has to pass the python layer which is now the bottleneck for the integration.

In order to speed up integration further we need to make sure the solver can evaluate the function and jacobian without calling into Python.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

Just to see that everything looks alright:

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 6))
cvode_sys.plot_result(tout, yout, info, ax=ax)
ax.set_xscale('log')
ax.set_yscale('log')